# **Chapter 6: Precognition (Thinking Step by Step)**

---
**Lesson:**

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to think through your answer first. 
Guess what? Claude is the same way.
Giving Claude time to think step by step sometimes makes Claude more accurate, particularly for complex tasks.

---

First we will setup our dependencies

In [2]:
%%capture
#Cell [1]

#langchain
%pip install langchain

#Install libraries 
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries
import json
import os
import sys
import boto3
import botocore

#Set up Bedrock runtime client
bedrock_runtime = boto3.client('bedrock-runtime')
boto3_bedrock = boto3.client('bedrock')

#Set up ModelID (Claude 3 Sonnet in this case)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

In [3]:
#Cell [2]

#Setup Model invocation function 
#Code modified from documentation: https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_Claude3_Text_section.html
def get_response(prompt_data):
    response = bedrock_runtime.invoke_model(
                    modelId=modelId,
                    body=json.dumps(
                        {
                            "anthropic_version": "bedrock-2023-05-31",
                            "max_tokens": 1024,
                            "messages": [
                                {
                                    "role": "user",
                                    "content": [{"type": "text", "text": prompt_data}],
                                }
                            ],
                        }
                    ),
                )
    result = json.loads(response.get("body").read())
    response = result.get("content", [])[0]['text']
    return response

# **Examples:**

**Example 6.1 - Movie Review**

In the movie review prompt below, it's clear to a human reader that the second sentence belies the first. But Claude takes the word "unrelated" too literally.

In [4]:
response = get_response("""

Human: Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since the year 1900.

Assistant:""")

print(response)

Based on the statement "This movie blew my mind with its freshness and originality", the sentiment expressed towards the movie in this review is positive. The phrase "blew my mind" implies the reviewer was highly impressed and found the movie to be very fresh and original. The additional statement about living under a rock seems unrelated to the movie review itself.


**Example 6.2 - Better movie review**

To improve Claude's response, let's allow Claude to think things out first before answering. We do that by literally spelling out the steps that Claude should take in order to process and think through the prompt. 
Now, Claude's response is much better and more nuanced.

In [5]:
response = get_response("""

Human: Is this review sentiment positive or negative? First write the best arguments for each side in <positive-arg> and <negative-arg> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900.

Assistant:""")

print(response)

<positive-arg>
- The reviewer describes the movie as fresh and original, which are positive qualities that suggest the movie is innovative and creative.
- Using the phrase "blew my mind" implies that the movie was exceptionally good and had a profound impact on the reviewer.
</positive-arg>

<negative-arg>
- The statement "I have been living under a rock since 1900" suggests that the reviewer may not have much context or basis for comparison when it comes to assessing movies, potentially undermining the credibility of their positive assessment.
- There is a lack of specific details or examples to support the positive claims about the movie's freshness and originality.
</negative-arg>

Overall, I would say this review sentiment is positive. While the second sentence raises some doubts about the reviewer's qualifications, the strong praise using phrases like "blew my mind" and describing the movie as fresh and original outweighs those concerns and indicates an overall positive sentiment 

**Example 6.3 - Blockbusters**

Letting Claude think can shift Claude's answer from incorrect to correct. It's that simple in many cases where Claude makes mistakes!
Let's go through an example where Claude's answer is incorrect to see how asking Claude to think can fix that.	

In [7]:
response = get_response("""

Human: Are both directors of Jaws and Casino Royale citizens of the same country?

Assistant:""")

print(response)

No, the directors of Jaws and Casino Royale are not citizens of the same country.

Jaws was directed by Steven Spielberg, who is an American citizen born in Cincinnati, Ohio, United States.

Casino Royale (the 2006 film starring Daniel Craig) was directed by Martin Campbell, who is a New Zealander born in Wanganui, New Zealand.

So while they are both acclaimed directors, Steven Spielberg is an American and Martin Campbell is a New Zealander, so they do not share citizenship of the same country.


**Example 6.4 - Blockbusters 2**

Let's fix this by asking Claude to think step by step. Instead of spelling out each step for Claude, you can also simply use those exact words and ask Claude to think step by step first.			

In [8]:
response = get_response("""

Human: Are both directors of Jaws and Casino Royale citizens of the same country? Make assumptions where needed, and think step by step.

Assistant:""")

print(response)

To answer this question, we need to determine the nationalities of the directors of the movies "Jaws" and "Casino Royale." Let's break it down step by step:

Step 1: Identify the director of "Jaws"
"Jaws" was directed by Steven Spielberg.

Step 2: Identify the nationality of Steven Spielberg
Steven Spielberg is an American director, producer, and screenwriter. He was born in Cincinnati, Ohio, USA.

Step 3: Identify the director of "Casino Royale" (2006 version)
The 2006 version of "Casino Royale" was directed by Martin Campbell.

Step 4: Identify the nationality of Martin Campbell
Martin Campbell is a New Zealand-born film director, producer, and writer. He was born in Wanganui, New Zealand, but later moved to the United Kingdom.

Step 5: Compare the nationalities
Steven Spielberg is an American citizen, while Martin Campbell holds New Zealand and British citizenship.

Based on the information available, the directors of "Jaws" (Steven Spielberg) and "Casino Royale" (2006) (Martin Camp

**Example 6.5 - Simon says**

It's important to note that to some degree, "let's think step by step" has been pre-built into Claude's training. You will sometimes see Claude thinking step by step even when you don't ask it to, particularly if you asked it a logic or math problem. However, even then, such techniques are not failproof, and Claude thinks step by step and still gets the wrong answer. To help Claude here, we can tell Claude exactly what process it should undertake to think through the problem.

In [9]:
response = get_response("""

Human: Use the following clues to answer the following multiple-choice question, using the following procedure:
(1) First, go through the clues one by one and consider whether the clue is potentially relevant
(2) Second, combine the relevant clues pay attention to every single word. And then reason out the answer to the question
(3) Third, map the answer to one of the multiple choice answers: either (a), (b), or (c)
 
Clues:
1. Miss Scarlett was the only person in the lounge.
2. The person with the pipe was in the kitchen.
3. Colonel Mustard was the only person in the observatory.
4. Professor Plum was not in the library nor the billiard room.
5. The person with the candlestick was in the observatory.
 
Question: Was Colonel Mustard in the observatory with the candlestick?
(a) Yes; Colonel Mustard was in the observatory with the candlestick
(b) No; Colonel Mustard was not in the observatory with the candlestick
(c) Unknown; there is not enough information to determine whether Colonel Mustard was in the observatory with the candlestick
 
Assistant:""")

print(response)

To solve this problem, let's follow the given procedure:

(1) Considering the relevance of each clue:
Clue 1: Potentially relevant, as it tells us where Miss Scarlett was.
Clue 2: Potentially relevant, as it gives information about the location of the person with the pipe.
Clue 3: Highly relevant, as it directly states where Colonel Mustard was.
Clue 4: Potentially relevant, as it tells us where Professor Plum was not.
Clue 5: Highly relevant, as it provides information about the location of the person with the candlestick.

(2) Combining the relevant clues:
From Clue 3, we know that "Colonel Mustard was the only person in the observatory."
From Clue 5, we know that "The person with the candlestick was in the observatory."

Combining these two clues, we can conclude that Colonel Mustard was the person in the observatory with the candlestick.

(3) Mapping the answer to the multiple-choice options:
Based on the reasoning above, the correct answer is (a) "Yes; Colonel Mustard was in the o

# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 6.1 - Classifying Emails**

In this exercise, we'll be instructing Claude to sort emails into the following categories:
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)

For the first part of the exercise, change the prompt in the cell below to make Claude output the correct classification. Your answer needs to include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category.

> *Tip: Use precognition and other techniques you've learned leading up to this chapter!*

In [11]:
#Create a prompt template that has input variable(s)
from langchain import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.") #Should be classified as (B)

response = get_response(prompt)

print(response)

Based on the details provided in the email, I would classify this as a product issue or complaint regarding a defective or malfunctioning product. The key points that indicate this classification are:

1. The email mentions a specific product, the "Mixmaster4000", which is likely a kitchen appliance like a mixer or blender.

2. It describes abnormal behavior from the product, such as producing a strange noise and a burning/smoky smell, which suggests a malfunction or defect.

3. The person explicitly states that they need a replacement for the product.

This type of email would typically be handled by a customer service or product support team, who would likely initiate a warranty claim, product replacement, or refund process for the defective item.


In [12]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?") #Should be classified as (A)

response = get_response(prompt)

print(response)

This email does not seem to contain any sensitive or confidential information. It appears to be a simple question asking whether a Mixmaster 4000 kitchen appliance can be used for mixing paint or if it is intended only for mixing food ingredients.

Given the context, this email can be classified as a general inquiry or product question and does not require any special handling or confidentiality measures.


In [13]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WHAT IS GOING ON???") #Should be classified as (C)

response = get_response(prompt)

print(response)

Based on the email content, this appears to be an angry customer complaint. The email is written in all capital letters, which often indicates the sender is expressing frustration or anger. The message mentions waiting 4 months for monthly charges to end after cancelling, implying an issue with cancellation of a service or subscription not being processed correctly. The use of multiple exclamation marks also conveys a sense of anger or exasperation.


In [14]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="How did I get here I am not good with computer.  Halp.") #Should be classified as (D)

response = get_response(prompt)

print(response)

Based on the text "How did I get here I am not good with computer. Halp.", this email seems to be a request for help or support related to computer or technology issues. The sender appears to be confused or unsure about how they ended up in a certain situation or interface, and they admit to not being skilled with computers, hence asking for assistance.

This type of email can be classified as a "Help/Support Request" or a "Technical Support Query." It is a common type of email sent by users who face difficulties or have questions regarding computer software, hardware, or other technological systems, and they seek guidance or solutions from a support team or a more knowledgeable individual.


# Chapter 6 - END.